In [2]:
import pandas as pd
df = pd.read_csv('/workspaces/threat_classification/scam_reports_sample_10000.csv')

In [2]:
df.head()

,pk_id,scam_id,scammer_id,description,createdOn,modifiedOn,scam_name,confirmed,scam_type,dollar_value,...,scammer_address_1,scammer_address_2,scammer_city,scammer_state,scammer_zip,scammer_phone,scammer_email,scammer_url,scam_type_id,definition
0,848147,848724,867691.0,received letter stating that I must call immed...,2024-05-30,2024-05-30,NaN,False,Phishing,0.0,...,NaN,NaN,NaN,NaN,NaN,(877) 205-9492,NaN,NaN,23,"In these schemes, scammers impersonating a tru..."
1,848330,848907,867874.0,Sent an email that read:Request Logged\nWe're ...,2024-05-30,2024-05-30,NaN,False,Other,345.0,...,NaN,NaN,NaN,NaN,NaN,1(111) 111-1111,sraylin_nhei2223@icloud.com,NaN,36,Another category of scam experience that curre...
2,848370,848947,867914.0,Email sent as fake invoice for Norton PC and t...,2024-05-30,2024-05-30,NaN,False,Phishing,0.0,...,NaN,NaN,NaN,NaN,NaN,(888) 462-4991,emberislacederada@gmail.com,NaN,23,"In these schemes, scammers impersonating a tru..."
3,848351,848928,867895.0,My husband and I was looking at renting a hous...,2024-05-30,2024-05-30,NaN,False,Online Purchase,0.0,...,NaN,NaN,Raeford,NC,NaN,NaN,NaN,Facebook Marketplace,43,These scams typically involve the purchase of ...
4,848001,848578,867545.0,Scammer charged to my credit card for somethin...,2024-05-30,2024-05-30,Online Purchase Scam,False,Online Purchase,0.0,...,NaN,NaN,D.C.,DC,NaN,(877) 653-1080,NaN,Vsquarecognicare.com,43,These scams typically involve the purchase of ...


In [13]:

print("max scams done to the same person: ", df[['pk_id', 'scammer_id']].groupby('pk_id').count().reset_index()['scammer_id'].max())
print("max scams by same scammer: ", df[['scammer_id', 'pk_id']].groupby('scammer_id').count().reset_index()['pk_id'].max())

max scams done to the same person:  2
max scams by same scammer:  2


In [15]:
df.columns

Index(['pk_id', 'scam_id', 'scammer_id', 'description', 'createdOn',
       'modifiedOn', 'scam_name', 'confirmed', 'scam_type', 'dollar_value',
       'target_state', 'target_zip', 'target_country', 'scammer_address_1',
       'scammer_address_2', 'scammer_city', 'scammer_state', 'scammer_zip',
       'scammer_phone', 'scammer_email', 'scammer_url', 'scam_type_id',
       'definition'],
      dtype='object')

In [63]:
df['scammer_phone'].isnull().mean()

np.float64(0.298)

In [78]:
#normalize phone number
import re
import phonenumbers
import numpy as np
df['scammer_phone'].iloc[0]
phonenumbers.parse(df['scammer_phone'].iloc[0], "US").national_number
def parse_phone_safe(x):
    try:
        return phonenumbers.parse(x, "US").national_number
    except:
        return x
df['scammer_phone'] = df['scammer_phone'].apply(lambda x: parse_phone_safe(x))


In [83]:
import pandas as pd
len(df['scammer_phone'].unique())
valid_rows = df[df['scammer_phone'].notnull() & df['scammer_email'].notnull()]
email_to_phone = valid_rows.set_index('scammer_email')['scammer_phone'].to_dict()
phone_to_email = valid_rows.set_index('scammer_phone')['scammer_email'].to_dict()
#where null and value availabil fill in 
df['scammer_phone'] = df.apply(lambda x: email_to_phone.get(x['scammer_email']) if pd.isnull(x['scammer_phone']) else x['scammer_phone'], axis=1)
df['scammer_email'] = df.apply(lambda x: phone_to_email.get(x['scammer_phone']) if pd.isnull(x['scammer_email']) else x['scammer_email'], axis=1)

In [90]:
df['description'].sample(10).iloc[0]

'They have sent me this letter several times stating my warranty was increasing in price April 4th was date n last letter I recieved. Asked me to call and use customer code# E2478571907 to get it set up for price increase.  I don&rsquo;t own a vehicle and haven&rsquo;t in 13+ yrs I&rsquo;m unable to drive I have epilepsy and brain tumor can&rsquo;t drive until 6 months seizure free and drs say never happen. I have not had license due to this in about 10 yrs now. I have called them telling them this before demanding to know who&rsquo;s got warranty in my name and for what car and they tell me it must have been mistake cause I&rsquo;m not in system.'

In [9]:
df['definition'][df['definition'].notnull()].apply(lambda x: len(x.split())).mean()

np.float64(46.93222053616591)

In [ ]:
hi